In [1]:
#Import earth engine
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=yk8C_cF_WZdxCDcEZ_3coseZ7wn-H7gufnftWKjXhK0&tc=z3hR9OXJn3XgHFJJdZVRnn2CTlonlFFgQ2UfQKWiHMs&cc=5rfW3_oS5l5uiwU9omk4XMHIY-SBPzngVKgc2y7eigg

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AZEOvhXvPJn9jiORbfmE61HsRsEvZizXfQ2AvtxjiRq7RdW0fa_xGQhMQ5A

Successfully saved authorization token.


In [2]:
dataset = ee.FeatureCollection('TIGER/2018/Counties')
visParams = {
    'palette': ['purple', 'blue', 'green', 'yellow', 'orange', 'red'],
    'min': 0,
    'max': 50,
    'opacity': 0.8,
}

In [3]:
dataset = dataset.map(lambda f: f.set('STATEFP', ee.Number.parse(f.get('STATEFP'))))
image = ee.Image().float().paint(dataset, 'STATEFP')

countyOutlines = ee.Image().float().paint(**{
    'featureCollection': dataset,
    'color': 'black',
    'width': 1
})

In [4]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[37.649, -99.844], zoom_start = 4)

# Add the layer to the map object.
my_map.add_ee_layer(image, visParams, 'TIGER/2018/Counties')
my_map.add_ee_layer(countyOutlines, {}, 'county outlines')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)